# Sensor Based Activity Recoginition 
Challenge: cdl1 - Sensor based Activity Recognition  
Team: Lea Bütler, Manjavy Kirupa, Etienne Roulet, Si Ben Tran  

Aufgabe: DL Modell erstellen

Hier in diesem Notebook erstellen wir unsere Deep Learning Modelle.

In [ ]:
import matplotlib.pyplot
import tensorflow as tf
from sklearn.model_selection import KFold
import numpy as np
import logging
from datetime import datetime
import dataclasses
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn import model_selection as ms
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import os
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import TensorBoard
import numpy as np

tf.debugging.set_log_device_placement(False)

# datetime as filename for logging
now = datetime.now()
date_time_string = now.strftime("%Y-%m-%d_%H-%M-%S")

logging.basicConfig(
    level=logging.INFO, filename=f"{date_time_string}.txt", filemode="a"
)
print(tf.config.list_physical_devices())

In [ ]:
%pip 

# Daten einlesen

In [ ]:
from dataclasses import dataclass


# Static Parameters
@dataclass
class Parameters:
    batch_size: int = 128
    epochs: int = 30
    verbosity: str = "auto"
    step_size: int = 374
    number_folds: int = 6
    output_size: int = 6
    window_size = 300
    step_size = 100
    data_augmentation  = True
    
    

In [ ]:
df = pd.read_csv("../Sensor_Data-Wrangling-und-EDA/Alle_Messungen_trimmed.csv")
df.head()

## Time umwandeln 

In [ ]:
# convert the string time column to datetime
epoch = pd.Timestamp("1970-01-01")

df["time"] = pd.to_datetime(df["time"])
df["time"] = (df["time"] - epoch).apply(lambda x: int(x.total_seconds() * 1000))

print(df.shape)

## Create Validation Data

In [ ]:
# select three random id_combines files
ids = ["06_iPhone12-2023-03-16_13-46-58Manjavy_KirupaVelofahren", "01_iPhone13pro-2023-03-21_16-55-47Etienne_RouletLaufen", "01_iPhone13ProMax-2023-03-15_18-29-42Gabriel_TorresRennen"]
df_validation = df[df["id_combined"].isin(ids)]

# export to csv 
df_validation.to_csv("validation-velo-laufen-rennen.csv", index=False)

# remove the validation data from the dataframe 
df = df[~df["id_combined"].isin(ids)]

## Drop unnötige Spalten und Class Encoden

In [ ]:
df = df.drop(columns=["id", "user", "id_combined"])
df.drop(["Unnamed: 0"], axis=1, inplace=True)
# get all types of the df
le = LabelEncoder()
df["class"] = le.fit_transform(df["class"])
# print dictionary of the classes and its encoded values
print(dict(zip(le.classes_, le.transform(le.classes_))))

## Daten Transformation

In [ ]:
import random
def add_noise(time_series, noise_factor):
    noise = np.random.randn(len(time_series)) * noise_factor
    return time_series + noise

# Flip time series
def flip_time_series(time_series):
    return np.flip(time_series)

# Scale magnitude
def scale_magnitude(time_series, scaling_factor):
    return time_series * scaling_factor

# Augment the data
def augment_data(X, y, noise_factor, scaling_factor, flip_probability):
    augmented_X = []
    augmented_y = []
    for i in range(X.shape[0]):
        x_sample = X[i]
        y_sample = y[i]

        # Add noise
        x_sample_noisy = add_noise(x_sample, noise_factor)
        augmented_X.append(x_sample_noisy)
        augmented_y.append(y_sample)

        # Scale magnitude
        x_sample_scaled = scale_magnitude(x_sample, scaling_factor)
        augmented_X.append(x_sample_scaled)
        augmented_y.append(y_sample)

        # Flip time series
        if random.random() < flip_probability:
            x_sample_flipped = flip_time_series(x_sample)
            augmented_X.append(x_sample_flipped)
            augmented_y.append(y_sample)

    return np.array(augmented_X), np.array(augmented_y)

In [ ]:
# Set the window size and step size

window_size = Parameters.window_size
step_size = Parameters.step_size
# Reshape X to 2D format (samples, features)
X = df.values[:, 1:13]
# Define y
y = df["class"].values
X_windows = np.array(
        [
            X[i : i + window_size, :]
            for i in range(0, X.shape[0] - window_size + 1, step_size)
        ]
    )

    # Reshape X_windows to 3D format (samples, timesteps, features)
timesteps = X_windows.shape[1]
n_features = X_windows.shape[2]
X_windows = X_windows.reshape(-1, timesteps, n_features)

if (Parameters.data_augmentation == False):

    # Create a sliding window of X with the specified window and step sizes
    

    # Create the corresponding y labels for the sliding windows
    y_windows = np.array(
        [y[i + window_size - 1] for i in range(0, X.shape[0] - window_size + 1, step_size)]
    )
    y_windows = to_categorical(y_windows, num_classes=6)

    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(
        X_windows, y_windows, test_size=0.1, random_state=42, stratify=y_windows
    )
elif (Parameters.data_augmentation == True):
        # Augmentation parameters
    noise_factor = 0.1
    scaling_factor = 1.5
    flip_probability = 0.5
    
    # Augment the data
    X_augmented, y_augmented = augment_data(X, y, noise_factor, scaling_factor, flip_probability)
    
    # Create sliding windows for the augmented data
    X_windows_augmented = np.array(
        [
            X_augmented[i : i + window_size, :]
            for i in range(0, X_augmented.shape[0] - window_size + 1, step_size)
        ]
    )
    X_windows_augmented = X_windows_augmented.reshape(-1, timesteps, n_features)
    
    y_windows_augmented = np.array(
        [y_augmented[i + window_size - 1] for i in range(0, X_augmented.shape[0] - window_size + 1, step_size)]
    )
    y_windows_augmented = to_categorical(y_windows_augmented, num_classes=6)
    # Split the dataset into training and testing sets
    X_train, X_validation, y_train, y_validaton = train_test_split(
        X_windows_augmented, y_windows_augmented, test_size=0.2, random_state=42, stratify=y_windows_augmented
    )
    

In [ ]:
X.shape, y.shape, X_train.shape, y_train.shape, X_validation.shape, y_validaton.shape

## Umschreibung

In [ ]:
x_train = X_train
x_validation = X_validation
y_train = y_train
y_validation = y_validaton

# DL-Modelle erstellen

In [ ]:
# Something like this as first Model

def create_model_1(name="model_1"):
    model = tf.keras.Sequential(
        [
            # Add a 1D convolutional layer
            tf.keras.layers.Conv1D(
                filters=64,
                kernel_size=12,
                activation="relu",
                padding="same",
                input_shape=(timesteps, n_features),
                kernel_regularizer=tf.keras.regularizers.l2(0.01),
            ),
            # Add LSTM layer
            tf.keras.layers.LSTM(100),
            # Add a dense output layer
            tf.keras.layers.Dense(
                6, activation="softmax"
            ),  # Change activation function based on the nature of the output
        ],
        name=name,
    )
    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy", tf.keras.metrics.Precision(), tf.keras.metrics.Recall()],
    )
    return model


def create_model_2(name="model_2"):
    model = tf.keras.Sequential(
        [
            # Add a 1D convolutional layer
            tf.keras.layers.Conv1D(
                filters=32,
                kernel_size=8,
                activation="relu",
                padding="same",
                input_shape=(timesteps, n_features),
                kernel_regularizer=tf.keras.regularizers.l2(0.001),
            ),
            # Add LSTM layer
            tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(300)),
            tf.keras.layers.Dropout(0.5),

            # Add a dense output layer
            tf.keras.layers.Dense(
                6, activation="softmax"
            ),  # Change activation function based on the nature of the output
        ],
        name=name,
    )
    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy", tf.keras.metrics.Precision(), tf.keras.metrics.Recall()],
    )
    return model

def create_model_3(name="model_3"):
    model = tf.keras.Sequential(
        [
            # Add a 1D convolutional layer
            tf.keras.layers.Conv1D(
                filters=32,
                kernel_size=2,
                activation="relu",
                padding="same",
                input_shape=(timesteps, n_features),
                kernel_regularizer=tf.keras.regularizers.l2(0.01),
            ),
            # Add LSTM layer
            tf.keras.layers.LSTM(100),
            # Add a dense output layer
            tf.keras.layers.Dense(
                6, activation="softmax"
            ),  # Change activation function based on the nature of the output
        ],
        name=name,
    )
    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy", tf.keras.metrics.Precision(), tf.keras.metrics.Recall()],
    )
    return model


def create_model_4(name="model_4"):
    model = tf.keras.Sequential(
        [
            # Add a 1D convolutional layer
            tf.keras.layers.Conv1D(
                filters=64,
                kernel_size=2,
                activation="relu",
                padding="same",
                input_shape=(timesteps, n_features),
                kernel_regularizer=tf.keras.regularizers.l2(0.01),
            ),
            # Add LSTM layer
            tf.keras.layers.LSTM(100),
            # Add a dense output layer
            tf.keras.layers.Dense(
                6, activation="softmax"
            ),  # Change activation function based on the nature of the output
        ],
        name=name,
    )
    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy", tf.keras.metrics.Precision(), tf.keras.metrics.Recall()],
    )
    return model


def create_model_5(name="model_5"):
    model = tf.keras.Sequential(
        [
            # Add a 1D convolutional layer
            tf.keras.layers.Conv1D(
                filters=32,
                kernel_size=2,
                activation="relu",
                padding="same",
                input_shape=(timesteps, n_features),
                kernel_regularizer=tf.keras.regularizers.l2(0.01),
            ),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPooling1D(pool_size=2),
            tf.keras.layers.Conv1D(
                filters=20, kernel_size=2, activation="relu", padding="same",
                kernel_regularizer=tf.keras.regularizers.l2(0.01),
            ),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPooling1D(pool_size=2),
            tf.keras.layers.Conv1D(
                filters=10, kernel_size=2, activation="relu", padding="same", kernel_regularizer=tf.keras.regularizers.l2(0.01),
            ),
            tf.keras.layers.MaxPooling1D(pool_size=2),
            # flatten output
            tf.keras.layers.Flatten(),
            # Add a dense output layer
            tf.keras.layers.Dense(180, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.01),),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dense(100, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.01),),
            tf.keras.layers.BatchNormalization(),
            # Change activation function based on the nature of the output
            tf.keras.layers.Dense(6, activation="softmax"),
        ],
        name=name,
    )

    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy", tf.keras.metrics.Precision(), tf.keras.metrics.Recall()],
    )
    return model

# DL-Modelle Trainieren

In [ ]:
best_model_history = None  # Keep track of the best model's history
model_histories = []

# Perform training and validation
models = [create_model_2]
best_model = None
best_model_acc = 0

for j, model_creator in enumerate(models):
    print(f"Model {model_creator.__name__}")
    model_name = f"Model_{j+1}"
    model = model_creator(name=model_name)
    logging.info(f"Model {j+1}")
    history = model.fit(
        X_train,
        y_train,
        epochs=Parameters.epochs,
        batch_size=Parameters.batch_size,
        validation_data=(x_validation, y_validation),
        verbose=Parameters.verbosity,
    )
    test_loss, acc, prec, recal = model.evaluate(
        x_validation, y_validation, verbose=Parameters.verbosity
    )
    logging.info(f"Validation accuracy: {acc}")

    model_histories.append(history.history)

    for epoch in range(Parameters.epochs):
        # Log accuracy after each epoch
        acc_epoch = history.history["val_accuracy"][epoch]
        logging.info(f"Epoch {epoch + 1}, Validation accuracy: {acc_epoch}")

    if best_model_history is None or acc > best_model_acc:
        best_model_history = history
        best_model = model  # Store the trained model instance
        best_model_acc = acc

print(best_model.name)


## Loss und Accuracy Verlauf vom besten Modell

In [ ]:
print(best_model.name)
display(best_model_history.history["accuracy"])
print(best_model_history.history["loss"])
# summarize history for accuracy
plt.plot(best_model_history.history["accuracy"])
plt.plot(best_model_history.history["val_accuracy"])
plt.title("model accuracy")
plt.ylabel("accuracy")
plt.xlabel("epoch")
plt.legend(["train", "test"], loc="upper left")
plt.show()
# summarize history for loss
plt.plot(best_model_history.history["loss"])
plt.plot(best_model_history.history["val_loss"])
plt.title("model loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.legend(["train", "test"], loc="upper left")
plt.show()

## Confusions Matrix

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# for multiclass classification
y_pred = best_model.predict(x_validation)
y_test_labels = y_validaton.argmax(axis=1)
y_pred_labels = y_pred.argmax(axis=1)

# create cm
cm = confusion_matrix(y_test_labels, y_pred_labels)
display(le.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=le.classes_)
# get f1 score of each class
from sklearn.metrics import f1_score

f1 = f1_score(y_test_labels, y_pred_labels, average=None)
# get f1 score of each class

# plot confusion matrix
disp.plot()
plt.show()
display(f1)

## Validation Daten Predicten

# Export bestes DL-Modell

In [ ]:
# export Model to json Tensorflow file
import json

best_model.save("saved_model/sensor_model.h5")

model = tf.keras.models.load_model("saved_model/sensor_model.h5")

# Save model architecture to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

# Save model weights to a JSON file
weights = model.get_weights()
weights_as_list = [w.tolist() for w in weights]
with open("weights.json", "w") as f:
    json.dump(weights_as_list, f)

# Modell Einlesen und predicten

In [ ]:

class validate_unseen_data():
    def __init__(self, model_path="saved_model/sensor_model.h5"):
        self.model = tf.keras.models.load_model(model_path)
        self.csv_path = "validation-velo-laufen-rennen.csv"
        
    def predict_classes(self, file="06_iPhone12-2023-03-16_13-46-58Manjavy_KirupaVelofahren", expected=''):
        df_val = pd.read_csv(self.csv_path)
        # Filter the rows where its velo in id_combined
        df_val = df_val[df_val["id_combined"].str.contains(file)]
        df_val = df_val.drop(columns=["id", "user", "id_combined"])
        # convert the string time column to datetime
        epoch = pd.Timestamp("1970-01-01")
        df_val["time"] = pd.to_datetime(df_val["time"])
        df_val["time"] = (df_val["time"] - epoch).apply(
            lambda x: int(x.total_seconds() * 1000)
        )
        df_val.drop(["Unnamed: 0"], axis=1, inplace=True)
        # get all types of the df
        le = LabelEncoder()
        df_val["class"] = le.fit_transform(df_val["class"])

        # Set the window size and step size
        window_size = 300
        step_size = 100

        # Reshape X to 2D format (samples, features)
        X = df_val.values[:, 1:13]

        # Create a sliding window of X with the specified window and step sizes
        X_windows = np.array(
            [
                X[i : i + window_size, :]
                for i in range(0, X.shape[0] - window_size + 1, step_size)
            ]
        )

        # Reshape X_windows to 3D format (samples, timesteps, features)
        timesteps = X_windows.shape[1]
        n_features = X_windows.shape[2]
        X_windows = X_windows.reshape(-1, timesteps, n_features)

        y_pred_probs = model.predict(X_windows)

        # Get the predicted class labels for each input window
        y_pred_labels = np.argmax(y_pred_probs, axis=1)

        # Print the predicted class labels
        # get the median of the predicted labels
        sol = np.median(y_pred_labels)

        class_counts = np.bincount(y_pred_labels)
        for i, count in enumerate(class_counts):
            print(f"Class {i} count: {count}")
        
        return (sol, expected)

median = validate_unseen_data().predict_classes('06_iPhone12-2023-03-16_13-46-58Manjavy_KirupaVelofahren', expected='velo')
median2 = validate_unseen_data().predict_classes('01_iPhone13ProMax-2023-03-15_18-29-42Gabriel_TorresRennen', expected='rennen')
median3 = validate_unseen_data().predict_classes('01_iPhone13pro-2023-03-21_16-55-47Etienne_RouletLaufen', expected='laufen')

(median, median2, median3)



In [ ]:
# the Conclusion is that the model is not overfitted but Gabriel is more a Stepper than a Runner ;)